In [123]:
using JLD2, FileIO
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

f = jldopen("../data/mixed_layer_simulation_Q-100_dTdz0.010_tau0.00_profiles.jld2","r+")
timeseries_keys = keys(f["timeseries"])

temp_horizantal_averages = length(f["timeseries"]["T"]["0"][:])

X = zeros(length(keys(f["timeseries"]["w"])),2,256)
Y = rand(Float32,(256,length(keys(f["timeseries"]["w"]))))

count = 1
for t_stamp in keys(f["timeseries"]["w"])
    X[count,1,:]=  f["timeseries"]["w"][t_stamp][:]   #aren't you supposed to append or iterate on X????
    X[count,2,:] = f["timeseries"]["wT"][t_stamp][:]
end

X = reshape(X,(length(keys(f["timeseries"]["w"])),512))'

m = Chain(
  Dense(256*2, 32, relu),
  Dense(32, 256),
  softmax)

loss(x, y) = Flux.mse(m(x), y)
opt = ADAM(); 
accuracy(x, y) = mean(Flux.mse(m(x), y))

dataset = repeated((X,Y),200) 
evalcb = () -> @show(loss(X, Y)) 
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10)); 

loss(X, Y) = 709.12354f0 (tracked)


In [124]:
# Training set accuracy
accuracy(X, Y)

0.005203816131830009